In [ ]:
import os
import xml
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

from nni.compression.pytorch import ModelSpeedup
from nni.compression.pytorch.utils.counter import count_flops_params

from utils import create_model, get_dataloader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

num_workers = 16
torch.set_num_threads(num_workers)

### Stanford Dogs (Data Download)

The [Stanford Dogs](http://vision.stanford.edu/aditya86/ImageNetDogs/) dataset contains images of 120 breeds of dogs from around the world. It is built using images and annotation from ImageNet for the task of fine-grained image classification. We choose this task to simulate a transfer learning scenario, where a model pre-trained on the ImageNet is further transferred to an often simpler downstream task.

To download and prepare the data, please run `prepare_data.sh`, which downloads the images and annotations, and preprocesses the images for training.

In [ ]:
# Run prepare_data.sh
!chmod u+x prepare_data.sh
!./prepare_data.sh

 Then, you may run following code block, which shows several instances:

# Model Pre-training
First, we obtain a MobileNetV2 model on this task, which will serve as the base model for compression. Unfortunately, although this step is often called model "pre-training" in the model compression teminologies, we are actually finetuning a model pre-trained on ImageNet. 

In [ ]:
%run pretrain.py --model_type mobilenet_v2_torchhub 

In [ ]:
%run pretrain.py --model_type resnet18

In [ ]:
%run pretrain.py --model_type efficientnet_se_b0

In [ ]:
%run pretrain.py --model_type efficientnet_b0 --n_epochs 100